In [ ]:
import json
import json
import os
import pandas as pd
import collections
import seaborn as sns
import math
import pyzipcode
import hashlib
from tqdm import tqdm

In [ ]:
chicago=True

In [ ]:
zipcodedb = pyzipcode.ZipCodeDatabase()

def get_zipcodes(states=["NY","CT","NJ","PA","VT"],use_chicago_zipcodes=False):
    if use_chicago_zipcodes:
        zips = ['60611', '60614', '60302', '60610', '60607', '60657', '60605',
                '60616', '60654', '60603', '60602', '60631', '60601', '60606',
                '60661', '60660', '60615', '60647', '60618', '60622', '60640',
                '60625', '60642', '60613', '60641', '60646', '60608', '60630',
                '60629', '60623', '60659', '60612', '60609', '60632', '60638',
                '60644', '60643', '60546']
    else:    
        zips = sum((zipcodedb.find_zip(state=state) for state in states), start=[])
        zips = [zipcode.zip for zipcode in zips]
    return zips


zipcodes = get_zipcodes(use_chicago_zipcodes=chicago)

In [ ]:
prod_mapping = pd.read_csv("../YelpZip-Data/productIdMapping.txt", delimiter="\t", names=["name","prod_id"])
yz_names = set(prod_mapping["name"])

In [ ]:
business_file = "../data/businesses/%s.json"

In [ ]:
def load_businesses(zipcode, loaded_ids):
    if not os.path.exists(business_file % zipcode):
        return
    with open(business_file % zipcode) as f:
        zipcode_data = json.load(f)
    for business in zipcode_data[zipcode]:
        bid = business["id"]
        if bid not in loaded_ids:
            loaded_ids.add(bid)
            yield business

In [ ]:
bids = set()
businesses = []
for zipcode in tqdm(zipcodes,total=len(zipcodes)):
    for business in load_businesses(zipcode, bids):
        if business["name"] in yz_names or chicago:
            businesses.append(business)

In [ ]:
len(businesses)

In [ ]:
bids2 = set()
businesses2 = []
for zipcode in tqdm(zipcodes,total=len(zipcodes)):
    for business in load_businesses(zipcode, bids2):
        businesses2.append(business)

In [ ]:
len(businesses2)

In [ ]:
businesses_by_zipcode = collections.defaultdict(list)
for business in businesses:
    businesses_by_zipcode[business["location"]["zip_code"]].append(business)

In [ ]:
try:
    os.mkdir("../data/businesses_by_zipcode/")
except:
    print("exists")
    pass
try:
    os.mkdir("../data/not_recommended_reviews/")
except:
    print("exists")
    pass
try:
    os.mkdir("../data/recommended_reviews/")
except:
    print("exists")
    pass

In [ ]:
for zipcode in businesses_by_zipcode:
    with open("../data/businesses_by_zipcode/%s.json" % zipcode, "w+") as f: json.dump(businesses_by_zipcode[zipcode], f)